- Update: 2023.03.13

# 0. 配置参数

- `dict_page`：品类链接的参数
- `out`：保存结果的路径

In [1]:
dict_page = {'url': 'https://www.dormanproducts.com/gsearch.aspx?type=keyword&origin=keyword&q=Turbocharger%2520Oil%2520Line&start=WHAT_THE_FUCK&num=100',
             'max_page': 2}

out = '../../file/dorman-oil_feed_line_assembly_2-menu.xlsx'

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import random
import time

In [4]:
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--incognito')
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36')
browser = webdriver.Chrome(options=chrome_options)

In [5]:
list_manufacturer = []
list_category = []
list_summary = []
list_url = []
list_href = []
for i in range(dict_page['max_page']):
    browser.get(dict_page['url'].replace('WHAT_THE_FUCK', str(i*100)))
    while len(browser.find_elements(by=By.CLASS_NAME, value='captcha-mid')) != 0:
        print('我尼玛反爬，换ip...')
        browser.quit()
        time.sleep(3)
        browser = webdriver.Chrome(options=chrome_options)
        browser.get(dict_page['url'].replace('WHAT_THE_FUCK', str(i*100)))
    
    print('爬取页面'+str(i+1)+'，实在是太6了！')
    list_article = browser.find_elements(by=By.XPATH, value='//*[@class="row searchResults"]')
    for article in list_article:
        img = article.find_element(by=By.CLASS_NAME, value='searchItems-img')
        list_url.append(img.find_element(by=By.TAG_NAME, value='a').get_attribute('href'))
        list_href.append(img.find_element(by=By.TAG_NAME, value='img').get_attribute('src'))
        info = article.find_element(by=By.CLASS_NAME, value='searchItems-info')
        list_manufacturer.append(info.find_element(by=By.TAG_NAME, value='h2').text)
        list_category.append(info.find_elements(by=By.TAG_NAME, value='div')[1].text)
        if len(info.find_elements(by=By.TAG_NAME, value='p')) == 0:
            summary = ''
        else:
            summary = info.find_elements(by=By.TAG_NAME, value='p')[0].text
        list_summary.append(summary)
    
    time.sleep(1)

browser.quit()

WebDriverException: Message: unknown error: net::ERR_PROXY_CONNECTION_FAILED
  (Session info: chrome=111.0.5563.65)
Stacktrace:
Backtrace:
	(No symbol) [0x00F1DCE3]
	(No symbol) [0x00EB39D1]
	(No symbol) [0x00DC4DA8]
	(No symbol) [0x00DC12F7]
	(No symbol) [0x00DB8835]
	(No symbol) [0x00DB980A]
	(No symbol) [0x00DB8B10]
	(No symbol) [0x00DB7F65]
	(No symbol) [0x00DB7EFC]
	(No symbol) [0x00DB6BF6]
	(No symbol) [0x00DB7358]
	(No symbol) [0x00DC6542]
	(No symbol) [0x00E1DB91]
	(No symbol) [0x00E0AECC]
	(No symbol) [0x00E1D57C]
	(No symbol) [0x00E0ACC6]
	(No symbol) [0x00DE6F68]
	(No symbol) [0x00DE80CD]
	GetHandleVerifier [0x01193832+2506274]
	GetHandleVerifier [0x011C9794+2727300]
	GetHandleVerifier [0x011CE36C+2746716]
	GetHandleVerifier [0x00FC6690+617600]
	(No symbol) [0x00EBC712]
	(No symbol) [0x00EC1FF8]
	(No symbol) [0x00EC20DB]
	(No symbol) [0x00ECC63B]
	BaseThreadInitThunk [0x76447D69+25]
	RtlInitializeExceptionChain [0x76F2BB9B+107]
	RtlClearBits [0x76F2BB1F+191]


In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame({'工厂号': list_manufacturer,
              '品类': list_category,
              '段落': list_summary,
              '链接': list_url,
              '图片': list_href}).to_excel(out, index=False)